In [3]:
import pandas as pd
merged_data = pd.read_csv("../../dataset/clean_feature/model_df.csv",index_col=0)

#### Standard Scaling

In [5]:
from sklearn.preprocessing import StandardScaler
columns_to_scale_merged = merged_data.columns.difference(['date', 'VKOSPI_Label'])

# 스탠다드 스케일링 진행
scaler = StandardScaler()
scaled_values_merged = scaler.fit_transform(merged_data[columns_to_scale_merged])

# 스케일링된 값을 원본 데이터프레임 'merged_data'의 해당 컬럼에 적용
merged_data[columns_to_scale_merged] = scaled_values_merged

Standard_Scled_Data = merged_data.to_csv("../../dataset/clean_feature/Standard_Sclaed_Data.csv")


### SVM 

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

# 1. Cross-Validation

X = merged_data.drop(columns=['VKOSPI_Label'])
y = merged_data['VKOSPI_Label']


# 5-fold SVM
cv_scores = cross_val_score(SVC(kernel='linear'), X, y, cv=5, scoring='accuracy')
cv_mean = cv_scores.mean()

cv_mean

0.6345454545454545

In [8]:
# 2. 하이퍼파라미터 튜닝

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

grid_search_cv = GridSearchCV(SVC(), param_grid, cv=5, n_jobs=-1)
grid_search_cv.fit(X, y)

best_params_cv = grid_search_cv.best_params_
best_score_cv = grid_search_cv.best_score_

best_params_cv, best_score_cv

({'C': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}, 0.7254545454545454)

In [9]:
# 3. Feature Importance
best_svm_model = SVC(kernel='linear', C=best_params_cv['C'], gamma=best_params_cv['gamma'])
best_svm_model.fit(X, y)

feature_importances = best_svm_model.coef_

# 데이터프레임으로 정리
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances[0]
}).sort_values(by='Importance', ascending=False)

feature_importance_df


,Feature,Importance
10,kospi_p_range,1.088974
15,NAS_day_fluc_range,0.738331
6,cny_night_change,0.445919
21,sp_day_change,0.402114
16,niv_night_change,0.276642
27,vix_day_fluc_range,0.195732
5,cny_yesterday_change,0.092880
25,vix_close_change,0.045611
3,cd_p_change(%),-0.043847
14,NAS_day_change,-0.065680


### Standard Scaling이 필요한 모델 분석
    * 로지스틱회귀
    * 서포트벡터머신
    * 신경망
    * K-최근접 이웃
    * 주성분 분석을 사용하는 모델

# 분석 개요

1. 모델 정의
2. CV를 통한 모델 성능 평가
3. 각 모델별 하이퍼파라미터 튜닝

##### 모델 정의 -> CV 성능평가

In [11]:
# 필요한 라이브러리 및 모듈 불러오기
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

# 모델 정의
models = {
    '로지스틱 회귀': LogisticRegression(max_iter=1000, random_state=42),
    '신경망': MLPClassifier(max_iter=1000, random_state=42),
    'k-최근접 이웃': KNeighborsClassifier(),
    'PCA로 주성분 분석 후 로지스틱 회귀': make_pipeline(PCA(n_components=10), LogisticRegression(max_iter=1000, random_state=42))
}

# 크로스 밸리데이션을 통한 모델 성능 평가
results = {}
for model_name, model in models.items():
    cv_score = cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()
    results[model_name] = cv_score

results

{'로지스틱 회귀': 0.6327272727272728,
 '신경망': 0.6854545454545453,
 'k-최근접 이웃': 0.6890909090909091,
 'PCA로 주성분 분석 후 로지스틱 회귀': 0.6890909090909091}

##### 하이퍼파라미터 튜닝

In [12]:
# 필요한 라이브러리 및 모듈 불러오기
from sklearn.model_selection import GridSearchCV

# 각 모델별 하이퍼파라미터 그리드 정의
param_grids = {
    '로지스틱 회귀': {
        'C': [0.001, 0.01, 0.1, 1, 10, 100]
    },
    '신경망': {
        'alpha': [0.0001, 0.001, 0.01, 0.1],
        'hidden_layer_sizes': [(50,), (100,), (50, 50)]
    },
    'k-최근접 이웃': {
        'n_neighbors': [3, 5, 7, 9, 11]
    },
    'PCA로 주성분 분석 후 로지스틱 회귀': {
        'pca__n_components': [5, 10, 15],
        'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100]
    }
}

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝 진행
tuned_parameters = {}
best_scores = {}

for model_name, model in models.items():
    grid_search = GridSearchCV(model, param_grids[model_name], cv=5, scoring='accuracy')
    grid_search.fit(X, y)
    
    # 최적의 하이퍼파라미터와 해당 정확도 저장
    tuned_parameters[model_name] = grid_search.best_params_
    best_scores[model_name] = grid_search.best_score_

tuned_parameters, best_scores


({'로지스틱 회귀': {'C': 0.1},
  '신경망': {'alpha': 0.1, 'hidden_layer_sizes': (100,)},
  'k-최근접 이웃': {'n_neighbors': 7},
  'PCA로 주성분 분석 후 로지스틱 회귀': {'logisticregression__C': 0.1,
   'pca__n_components': 5}},
 {'로지스틱 회귀': 0.6872727272727273,
  '신경망': 0.7036363636363635,
  'k-최근접 이웃': 0.7236363636363636,
  'PCA로 주성분 분석 후 로지스틱 회귀': 0.7418181818181818})

##### 로지스틱 회귀분석만 피처셀렉 (임베디드 방법 : SelectFromModel)
* 신경망, k-최근접 이웃 : 접적인 특성 중요도를 제공하지 않음
* PCA로 주성분 분석 후 로지스틱 회귀: PCA 자체가 차원 축소 기법이므로 추가적인 특성 선택은 생략

In [13]:
# 필요한 모듈 불러오기
from sklearn.feature_selection import SelectFromModel

# 로지스틱 회귀를 위한 특성 선택
selector = SelectFromModel(estimator=LogisticRegression(max_iter=1000, random_state=42)).fit(X, y)
X_selected = selector.transform(X)

# 특성 선택 후의 모델 정의
models_selected = {
    '로지스틱 회귀 (선택된 특성)': LogisticRegression(max_iter=1000, random_state=42)
}

# 크로스 밸리데이션을 통한 모델 성능 평가
results_selected = {}
for model_name, model in models_selected.items():
    cv_score = cross_val_score(model, X_selected, y, cv=5, scoring='accuracy').mean()
    results_selected[model_name] = cv_score
    print(f"{model_name}: 정확도 = {cv_score:.4f}")

results_selected


로지스틱 회귀 (선택된 특성): 정확도 = 0.7236


{'로지스틱 회귀 (선택된 특성)': 0.7236363636363636}

##### 로지스틱 회귀계수에 따른 피처 중요도

In [14]:
# 모든 특성을 사용하여 로지스틱 회귀 모델 학습
logreg = LogisticRegression(max_iter=1000, random_state=42).fit(X, y)

# 선택된 특성의 이름 가져오기
selected_features = X.columns[selector.get_support()]

# 선택된 특성에 대한 계수 가져오기
selected_coef = logreg.coef_[0][selector.get_support()]

# 특성 중요도를 표시하기 위한 DataFrame 생성
feature_importance = pd.DataFrame({
    '특성': selected_features,
    '계수 (중요도)': selected_coef
}).sort_values(by='계수 (중요도)', ascending=False)

feature_importance


,특성,계수 (중요도)
5,kospi_p_range,1.118065
9,sp_day_change,0.513973
12,vix_close_change,0.496348
0,basis,-0.383437
7,NAS_close_change,-0.398166
8,put_volume_fluc,-0.399763
11,usd_yesterday_change,-0.435076
2,basis_disparate_ratio,-0.481379
3,jpy_yesterday_change,-0.530402
1,basis_risk,-0.680370


In [15]:
from sklearn.feature_selection import RFECV

# 한글 주석으로 코드 정리
selected_features_by_wrapper = {}

# 로지스틱 회귀에 대한 래퍼 방법 적용
selector_logreg = RFECV(estimator=LogisticRegression(max_iter=1000, random_state=42), step=1, cv=5).fit(X, y)
selected_features_by_wrapper['로지스틱 회귀'] = X.columns[selector_logreg.support_]

# 신경망에 대한 래퍼 방법 적용
# 신경망은 학습 시간이 오래 걸릴 수 있으므로 여기서는 생략하겠습니다.

# k-최근접 이웃에 대한 래퍼 방법 적용
# k-최근접 이웃은 느린 계산 속도로 인해 RFECV에 부적합할 수 있으므로 여기서는 생략하겠습니다.

selected_features_by_wrapper

{'로지스틱 회귀': Index(['basis_disparate_ratio', 'kospi_p_range', 'ks200_p_change(%)',
        'put_volume_fluc'],
       dtype='object')}

In [16]:
# 선택된 특성으로 데이터셋 준비
X_selected_logreg = X[selected_features_by_wrapper['로지스틱 회귀']]

# 로지스틱 회귀 모델 학습 및 평가
logreg_selected_score = cross_val_score(LogisticRegression(max_iter=1000, random_state=42), 
                                        X_selected_logreg, y, cv=5, scoring='accuracy').mean()

logreg_selected_score


0.7981818181818181